In [75]:
import pandas as pd
from pybiomart import Server
import numpy as np
import re
from io import StringIO

## Find the intersection genes between training and testing datasets

In [92]:
# Read Gene from the Train data
file_path = "GSE282742_TPM.txt"
train_df = pd.read_csv(file_path, sep='\t')
# display(train_df.head(5))
train_genes = train_df['gene_id'].tolist()
print(f"Genes in Train Data: {len(train_genes)}")

# Read Gene from the Test data
test_file_path = 'GSE249477_raw_count_normalize_04-10-2025.csv' 
test_df = pd.read_csv(test_file_path)
# display(test_df.head(5))
gene_test_df = test_df[['Identifier', 'Name']].copy()
test_genes = gene_test_df['Identifier'].tolist()
print(f"Genes in Test Data: {len(test_genes)}")

# Get the intersection genes
intersection_genes = list(set(train_genes) & set(test_genes))
print(f"Intersection genes: {len(intersection_genes)}")
genes_id_and_symbol = gene_test_df[gene_test_df['Identifier'].isin(intersection_genes)].reset_index(drop=True)
genes_id_and_symbol = genes_id_and_symbol.rename(columns={'Identifier':'gene_id','Name':'gene_symbol'})
print(genes_id_and_symbol.shape)
display(genes_id_and_symbol.head(5))

Genes in Train Data: 61860
Genes in Test Data: 21492
Intersection genes: 21462
(21462, 2)


/var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/ipykernel_32088/2899486901.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(test_file_path)


,gene_id,gene_symbol
0,ENSG00000186092,OR4F5
1,ENSG00000284733,OR4F29
2,ENSG00000284662,OR4F16
3,ENSG00000187634,SAMD11
4,ENSG00000188976,NOC2L


In [93]:
# Save the DataFrame "genes_id_and_symbolrame" to a CSV file
genes_id_and_symbol.to_csv('gene_id_and_gene_symbols.csv', index=False)

In [94]:
# Check valid gene symbols
original_list = genes_id_and_symbol['gene_symbol'].unique().tolist()

# Filter the list
filtered_list = [item for item in original_list if len(item) >= 10]

print(filtered_list)

['gene:ENSG00000225931', 'gene:ENSG00000279839', 'gene:ENSG00000285629', 'gene:ENSG00000280113', 'CENPS-CORT', 'gene:ENSG00000288636', 'gene:ENSG00000280222', 'MICOS10-NBL1', 'gene:ENSG00000279625', 'gene:ENSG00000255054', 'gene:ENSG00000279443', 'gene:ENSG00000288678', 'gene:ENSG00000254553', 'gene:ENSG00000278966', 'gene:ENSG00000278997', 'gene:ENSG00000279179', 'gene:ENSG00000284773', 'gene:ENSG00000271741', 'gene:ENSG00000274944', 'gene:ENSG00000279667', 'gene:ENSG00000284895', 'gene:ENSG00000283580', 'gene:ENSG00000284989', 'gene:ENSG00000288208', 'P3R3URF-PIK3R3', 'gene:ENSG00000279096', 'gene:ENSG00000279214', 'gene:ENSG00000279324', 'gene:ENSG00000285839', 'gene:ENSG00000280378', 'gene:ENSG00000256407', 'MROH7-TTC4', 'gene:ENSG00000284686', 'gene:ENSG00000278967', 'gene:ENSG00000280317', 'FPGT-TNNI3K', 'ST6GALNAC3', 'ST6GALNAC5', 'gene:ENSG00000280099', 'gene:ENSG00000267561', 'gene:ENSG00000279778', 'gene:ENSG00000288629', 'gene:ENSG00000271949', 'TLCD4-RWDD3', 'gene:ENSG00000

## Normalization and Filtering High Information

In [95]:
def normalize_and_filter_tpm(df_input: pd.DataFrame, mean_threshold: float = 0.5, std_threshold: float = 0.8) -> pd.DataFrame:
    """
    Normalizes TPM data using log2(TPM + 1) and filters genes based on mean 
    and standard deviation across samples.

    The input DataFrame MUST have Samples as rows and Genes as columns.

    Args:
        df_input (pd.DataFrame): DataFrame with rows=samples, columns=genes (raw TPM).
        mean_threshold (float): Genes with mean log2(TPM+1) < this value are filtered out.
        std_threshold (float): Genes with standard deviation < this value are filtered out.

    Returns:
        pd.DataFrame: Normalized and filtered DataFrame.
    """
    # 1. Normalization: log2(TPM + 1) transformation
    # This stabilizes variance and makes the data more symmetric.
    df_normalized = np.log2(df_input + 1)

    # 2. Calculate mean and standard deviation for each gene (column) across all samples (rows)
    # axis=0 means calculations are performed down the rows (across samples) for each column (gene).
    gene_means = df_normalized.mean(axis=0)
    gene_stds = df_normalized.std(axis=0)

    # 3. Filtering: Identify genes (columns) to keep
    # Keep gene if (mean >= threshold AND std >= threshold)
    genes_to_keep_mask = (gene_means >= mean_threshold) & (gene_stds >= std_threshold)
    
    genes_to_keep = gene_means.index[genes_to_keep_mask]
    
    # 4. Filter the DataFrame
    df_filtered = df_normalized[genes_to_keep]
    
    return df_filtered

In [154]:
# Training normalization
filtered_train_df = train_df[train_df['gene_id'].isin(intersection_genes)].reset_index(drop=True)
filtered_train_df = filtered_train_df.rename(columns={'gene_id':'samples'})
filtered_train_df = filtered_train_df.set_index('samples')
transposed_train_df = filtered_train_df.T
print(f"Filtered Train Data shape: {transposed_train_df.shape}") 
# display(transposed_train_df.head(5))

normalized_train_df = normalize_and_filter_tpm(transposed_train_df)
print(f"Normalized Train Data shape: {normalized_train_df.shape}") 
display(normalized_train_df.head(5))

Filtered Train Data shape: (116, 21462)
Normalized Train Data shape: (116, 450)


samples,ENSG00000115590,ENSG00000198502,ENSG00000182557,ENSG00000260537,ENSG00000112759,ENSG00000205810,ENSG00000278588,ENSG00000280071,ENSG00000239951,ENSG00000019169,...,ENSG00000184357,ENSG00000137959,ENSG00000197353,ENSG00000211945,ENSG00000270149,ENSG00000276410,ENSG00000275743,ENSG00000179044,ENSG00000244437,ENSG00000183134
VGH0075,7.328585,7.327867,2.510962,0.000000,4.150560,4.136684,0.000000,0.000000,5.380245,2.794936,...,0.000000,7.244697,2.063503,4.263786,3.724650,0.000000,2.422233,2.990955,4.721919,2.906891
VGH0089,7.129901,0.516015,1.286881,1.541019,2.217231,4.126808,0.367371,4.344118,5.435629,2.718088,...,0.650765,3.695994,3.070389,2.885574,3.738768,0.887525,1.541019,1.664483,4.575917,1.070389
VGH0146,5.891176,3.903038,2.010780,0.782409,2.601697,3.732269,0.526069,4.614710,5.325890,1.035624,...,0.286881,3.667892,3.786596,3.909773,2.017922,0.000000,2.831877,2.244887,4.754888,2.144046
VGH0195,6.367196,4.372256,2.799087,0.000000,4.012569,2.641546,1.070389,4.995485,6.419707,3.904966,...,1.389567,4.786074,4.426265,5.081936,2.356144,1.327687,3.145677,2.217231,5.572890,2.601697
VGH0203,9.639413,3.112700,1.646163,2.000000,2.526069,2.117695,0.575312,4.408032,4.620000,4.241840,...,0.565597,1.996389,1.049631,2.941106,2.695994,0.565597,3.375735,1.372952,3.764474,0.799087


In [101]:
gene_cohort = normalized_train_df.columns.tolist()
print(f"Genes in cohort: {len(gene_cohort)}")

Genes in cohort: 450


In [128]:
# Testing normalization
filtered_test_df = test_df[test_df['Identifier'].isin(gene_cohort)].reset_index(drop=True)

gene_id_column = 'Identifier' 
test_col_list = filtered_test_df.columns.tolist()
regex_pattern = r".*TPM.*"
gene_expressions = [item for item in test_col_list if re.search(regex_pattern, item, re.IGNORECASE)]
columns_to_select = [gene_id_column] + gene_expressions
test_subset_df = filtered_test_df[columns_to_select].copy()
test_subset_df = test_subset_df.rename(columns={'Identifier':'samples'})
test_subset_df = test_subset_df.set_index('samples')
transposed_test_df = test_subset_df.T
transposed_test_df.index = transposed_test_df.index.str.split(' ').str[0]
display(transposed_test_df)

normalized_test_df = np.log2(transposed_test_df + 1)
print(f"Normalized Test Data shape: {normalized_test_df.shape}") 
display(normalized_test_df.head(5))

samples,ENSG00000188290,ENSG00000187608,ENSG00000188157,ENSG00000049247,ENSG00000176083,ENSG00000126709,ENSG00000168528,ENSG00000162366,ENSG00000162383,ENSG00000177606,...,ENSG00000007350,ENSG00000288258,ENSG00000288709,ENSG00000129824,ENSG00000067646,ENSG00000114374,ENSG00000067048,ENSG00000183878,ENSG00000012817,ENSG00000198692
DK22119_01,0.000000,10.362002,0.000000,1.151334,6.908002,94.409354,2.302667,171.548705,2.302667,6.908002,...,17.270004,1.151334,19.572671,78.290684,4.605334,18.421337,42.599343,17.270004,14.967337,71.382683
DK22119_02,2.205215,48.514732,2.205215,14.333898,4.410430,372.681354,2.205215,191.853715,3.307823,16.539113,...,3.307823,8.820860,0.000000,103.645110,2.205215,26.462581,49.617340,40.796480,12.128683,65.053846
DK22119_03,0.000000,27.395290,1.141470,9.131763,25.112349,170.079093,4.565882,279.660253,4.565882,21.687938,...,0.000000,0.000000,12.556175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.141470
DK22119_04,1.003041,133.404482,2.006082,8.024330,12.036495,700.122772,2.006082,163.495719,8.024330,9.027371,...,3.009124,4.012165,0.000000,151.459224,6.018247,31.094278,64.194638,29.088195,5.015206,44.133814
DK22119_05,1.374805,26.121291,0.000000,6.874024,6.874024,112.733992,2.749610,268.086932,1.374805,16.497657,...,0.000000,1.374805,9.623633,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DK22119_59,0.000000,16.241439,1.624144,8.120719,17.865583,121.810790,3.248288,354.063364,3.248288,9.744863,...,1.624144,3.248288,0.000000,82.831338,25.986302,29.234590,30.858734,21.113870,4.872432,50.348460
DK22119_60,9.609882,338.481382,2.135529,0.000000,21.355292,717.537820,5.338823,343.820205,4.271058,6.406588,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DK22119_61,0.000000,66.812169,3.930128,2.620085,3.930128,340.611056,0.000000,345.851226,1.310043,17.030553,...,0.000000,3.930128,14.410468,49.781616,1.310043,13.100425,20.960680,6.550213,2.620085,17.030553
DK22119_62,4.050228,70.203956,2.700152,10.800609,24.301369,234.913237,5.400304,413.123280,9.450533,17.550989,...,0.000000,4.050228,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Normalized Test Data shape: (62, 450)


samples,ENSG00000188290,ENSG00000187608,ENSG00000188157,ENSG00000049247,ENSG00000176083,ENSG00000126709,ENSG00000168528,ENSG00000162366,ENSG00000162383,ENSG00000177606,...,ENSG00000007350,ENSG00000288258,ENSG00000288709,ENSG00000129824,ENSG00000067646,ENSG00000114374,ENSG00000067048,ENSG00000183878,ENSG00000012817,ENSG00000198692
DK22119_01,0.000000,3.506145,0.000000,1.105231,2.983313,6.576059,1.723632,7.430860,1.723632,2.983313,...,4.191405,1.105231,4.362657,6.309079,2.486800,4.279571,5.446234,4.191405,3.997052,6.177573
DK22119_02,1.680421,5.629786,1.680421,3.938653,2.435743,8.545665,1.680421,7.591363,2.106959,4.132504,...,2.106959,3.295849,0.000000,6.709361,1.680421,4.779395,5.661560,5.385310,3.714650,6.045571
DK22119_03,0.000000,4.827580,1.098602,3.340813,4.706660,7.418520,2.476610,8.132681,2.476610,4.503854,...,0.000000,0.000000,3.760878,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.098602
DK22119_04,1.002192,7.070437,1.587885,3.173820,3.704484,9.453523,1.587885,7.361906,3.173820,3.325871,...,2.003287,2.325434,0.000000,7.252280,2.811111,5.004244,6.026681,4.911126,2.588614,5.496137
DK22119_05,1.247809,4.761354,0.000000,2.977101,2.977101,6.829520,1.906740,8.071929,1.247809,4.129090,...,0.000000,1.247809,3.409205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [150]:
# Check gene cohort in both trainig and testing
train_gene_cohort = set(normalized_train_df.columns.tolist())
test_gene_cohort = set(normalized_test_df.columns.tolist())

intersect = list(train_gene_cohort & test_gene_cohort)
print(f"There are {len(intersect)} genes overlap")

There are 450 genes overlap


### Test data pre-processing 

- original genes: 21492 genes
- gene cohort: 450 genes

In [129]:
# Preprocess Test Series Matrix
def extract_geo_metadata_to_dataframe(file_path):
    """
    Parses metadata lines (!Sample_title, !Sample_description, 
    !Sample_characteristics_ch1) from a GEO Series Matrix file content 
    and returns a pandas DataFrame.
    """
    
    # 1. Initialize lists to hold the data
    metadata = {}
    
    # 2. Define the exact line prefixes we are interested in
    TARGET_LINES = [
        "!Sample_geo_accession", 
        "!Sample_title",
        "!Sample_description",
        "!Sample_characteristics_ch1"
    ]
    
    # 3. Read content line by line
    with open(file_path, 'r') as f:
        file_content = f.read()

    # 4. Process each line to extract relevant metadata
    for line in file_content.split('\n'):
        # Check if the line is one of our target metadata lines and is not blank
        if line.startswith(tuple(TARGET_LINES)):
            # Split the line by the first tab to separate the field name from the values
            parts = line.split('\t', 1)
            
            if len(parts) < 2:
                continue
            
            field_name = parts[0]
            raw_values = parts[1]
            
            # Use regex to strip the quotes and split by tab
            # This handles cases where values themselves contain spaces
            values = re.findall(r'"(.*?)"', raw_values)
            
            if not values:
                continue

            # Store the Sample Accession IDs (GSMs) separately for column headers
            if field_name == "!Sample_geo_accession":
                sample_ids = values
                metadata['Sample_ID'] = sample_ids
            
            # Handle Sample Characteristics (multiple rows with key: value pairs)
            elif field_name == "!Sample_characteristics_ch1":
                # The characteristic is always stored as "key: value" (e.g., "age: 81y.o.")
                # We need to split the key and value and create new dictionary entries
                
                # The first sample determines the key structure
                if ':' in values[0]:
                    key_value = values[0].split(':', 1)
                    key = key_value[0].strip().replace(' ', '_')
                else:
                    # Skip if the characteristic line doesn't conform (e.g., just 'blood')
                    continue 

                # Store the key-value pairs. Since keys can repeat (e.g., 'tissue', 'disease state', 'age', 'Sex'),
                # we use the value of the first element of the pair as the unique column name
                # E.g., 'disease state' -> 'disease state'
                #       'disease state: Alzheimer\'s disease' -> 'disease state'
                # We use the full label (e.g., 'disease state') as the column name for clarity.
                
                if key not in metadata:
                    metadata[key] = []
                    
                for val in values:
                    # Extract the value part after the first colon
                    try:
                        if key == 'age':
                            # Special handling for age to remove 'y.o.' suffix
                            value_only = val.split(':', 1)[1].strip().replace('y.o.', '').strip()
                        else:
                            value_only = val.split(':', 1)[1].strip()
                        metadata[key].append(value_only)
                    except IndexError:
                        # Handle cases where value might be missing the colon separator
                        metadata[key].append(val)
            
            # Handle single-row metadata like Sample_title and Sample_description
            else:
                # Clean the field name for the column header (e.g., remove '!' and use 'Sample_title')
                col_name = field_name.replace('!', '')
                metadata[col_name] = values
    print(metadata)
    # 5. Convert the collected dictionary into a DataFrame
    # If the Sample_ID list exists, use it for the index
    if 'Sample_ID' in metadata and metadata['Sample_ID']:
        
        # Prepare data for DataFrame, ensuring all lists have the same length as sample_ids
        data_for_df = {}
        expected_len = len(metadata['Sample_ID'])
        
        for key, val_list in metadata.items():
            if key != 'Sample_ID' and len(val_list) == expected_len:
                data_for_df[key] = val_list

        # Create DataFrame
        df = pd.DataFrame(data_for_df, index=metadata['Sample_ID'])
        df.index.name = "Sample_ID"
        return df.reset_index()

    return pd.DataFrame()

test_series_matrix_file = 'GSE249477_series_matrix.txt'
metadata_df = extract_geo_metadata_to_dataframe(test_series_matrix_file)
metadata_df = metadata_df[['Sample_description','disease_state','age','Sex']]
metadata_df = metadata_df.rename(columns={'Sample_description':'samples'})
metadata_df = metadata_df.set_index('samples')

print("--- Sample Metadata DataFrame ---")
display(metadata_df.head())
print(f"\nDataFrame shape: {metadata_df.shape}")


{'Sample_title': ["blood, Alzheimer's disease, 81y.o., male [DK22119_01 ]", "blood, Alzheimer's disease, 80y.o., male [DK22119_02]", "blood, Alzheimer's disease, 79y.o., female [DK22119_03]", "blood, Alzheimer's disease, 80y.o., male [DK22119_04]", "blood, Alzheimer's disease, 83y.o., female [DK22119_05]", "blood, Alzheimer's disease, 90y.o., male [DK22119_06]", "blood, Alzheimer's disease, 91y.o., female [DK22119_07]", "blood, Alzheimer's disease, 91y.o., female [DK22119_08]", "blood, Alzheimer's disease, 79y.o., female [DK22119_09]", "blood, Alzheimer's disease, 83y.o., male [DK22119_10]", "blood, Alzheimer's disease, 82y.o., male [DK22119_11]", "blood, Alzheimer's disease, 87y.o., male [DK22119_12]", "blood, Alzheimer's disease, 88y.o., female [DK22119_13]", "blood, Alzheimer's disease, 83y.o., female [DK22119_14]", "blood, Alzheimer's disease, 83y.o., male [DK22119_15]", "blood, Alzheimer's disease, 90y.o., female [DK22119_16]", "blood, Alzheimer's disease, 93y.o., female [DK22119_

,disease_state,age,Sex
samples,,,
DK22119_01,Alzheimer's disease,81,male
DK22119_02,Alzheimer's disease,80,male
DK22119_03,Alzheimer's disease,79,female
DK22119_04,Alzheimer's disease,80,male
DK22119_05,Alzheimer's disease,83,female



DataFrame shape: (62, 3)


In [141]:
# Merge genes and metadata
test_genes_metadata = pd.merge(
    metadata_df,
    normalized_test_df,
    left_index=True,   # Use the index of normalized_test_df
    right_index=True,  # Use the index of metadata_df
    how='inner'        # Keep only samples present in both
)
test_genes_metadata.head(5)

print(test_genes_metadata['disease_state'].unique())
print(test_genes_metadata['Sex'].unique())


["Alzheimer's disease"
 "mild cognitive impairment due to Alzheimer's disease"
 'cognitively normal control']
['male' 'female']


In [142]:
# Change values
disease_state_mapping_dict = {"Alzheimer's disease": 'AD', 
                              "mild cognitive impairment due to Alzheimer's disease": 'MCI',
                              "cognitively normal control":'C'}
test_genes_metadata['disease_state'] = test_genes_metadata['disease_state'].map(disease_state_mapping_dict)

sex_mapping_dict = {"male":'M', "female":'F'}
test_genes_metadata['Sex'] = test_genes_metadata['Sex'].map(sex_mapping_dict)

print(test_genes_metadata['disease_state'].unique())
print(test_genes_metadata['Sex'].unique())
# Drop the control samples
final_test_genes_metadata = test_genes_metadata[(test_genes_metadata['disease_state'] != 'C')]
print(f"Final test data: {final_test_genes_metadata.shape}")

['AD' 'MCI' 'C']
['M' 'F']
Final test data: (41, 453)


In [143]:
# Save the DataFrame "test" to a CSV file
final_test_genes_metadata.to_csv('test_normalized_data.csv', index=True)

### Train data pre-processing

In [155]:
# Preprocess Train Series Matrix
def extract_geo_metadata_to_dataframe(file_path):
    """
    Parses metadata lines (!Sample_title, !Sample_description, 
    !Sample_characteristics_ch1) from a GEO Series Matrix file content 
    and returns a pandas DataFrame.
    """
    
    # 1. Initialize lists to hold the data
    metadata = {}
    
    # 2. Define the exact line prefixes we are interested in
    TARGET_LINES = [
        "!Sample_geo_accession", 
        "!Sample_title",
        "!Sample_description",
        "!Sample_characteristics_ch1"
    ]
    
    # 3. Read content line by line
    with open(file_path, 'r') as f:
        file_content = f.read()

    # 4. Process each line to extract relevant metadata
    for line in file_content.split('\n'):
        # Check if the line is one of our target metadata lines and is not blank
        if line.startswith(tuple(TARGET_LINES)):
            # Split the line by the first tab to separate the field name from the values
            parts = line.split('\t', 1)
            
            if len(parts) < 2:
                continue
            
            field_name = parts[0]
            raw_values = parts[1]
            
            # Use regex to strip the quotes and split by tab
            # This handles cases where values themselves contain spaces
            values = re.findall(r'"(.*?)"', raw_values)
            
            if not values:
                continue

            # Store the Sample Accession IDs (GSMs) separately for column headers
            if field_name == "!Sample_geo_accession":
                sample_ids = values
                metadata['Sample_ID'] = sample_ids
            
            # Handle Sample Characteristics (multiple rows with key: value pairs)
            elif field_name == "!Sample_characteristics_ch1" or field_name == "!Sample_description":
                # The characteristic is always stored as "key: value" (e.g., "age: 81y.o.")
                # We need to split the key and value and create new dictionary entries
                
                # The first sample determines the key structure
                if ':' in values[0]:
                    key_value = values[0].split(':', 1)
                    key = key_value[0].strip().replace(' ', '_')
                else:
                    # Skip if the characteristic line doesn't conform (e.g., just 'blood')
                    continue 

                # Store the key-value pairs. Since keys can repeat (e.g., 'tissue', 'disease state', 'age', 'Sex'),
                # we use the value of the first element of the pair as the unique column name
                # E.g., 'disease state' -> 'disease state'
                #       'disease state: Alzheimer\'s disease' -> 'disease state'
                # We use the full label (e.g., 'disease state') as the column name for clarity.
                
                if key not in metadata:
                    metadata[key] = []
                    
                for val in values:
                    # Extract the value part after the first colon
                    try:
                        if key == 'age':
                            # Special handling for age to remove 'y.o.' suffix
                            value_only = val.split(':', 1)[1].strip().replace('y', '').strip()
                        else:
                            value_only = val.split(':', 1)[1].strip()
                        metadata[key].append(value_only)
                    except IndexError:
                        # Handle cases where value might be missing the colon separator
                        metadata[key].append(val)
            
            # Handle single-row metadata like Sample_title and Sample_description
            else:
                # Clean the field name for the column header (e.g., remove '!' and use 'Sample_title')
                col_name = field_name.replace('!', '')
                metadata[col_name] = values
    print(metadata)
    # 5. Convert the collected dictionary into a DataFrame
    # If the Sample_ID list exists, use it for the index
    if 'Sample_ID' in metadata and metadata['Sample_ID']:
        
        # Prepare data for DataFrame, ensuring all lists have the same length as sample_ids
        data_for_df = {}
        expected_len = len(metadata['Sample_ID'])
        
        for key, val_list in metadata.items():
            if key != 'Sample_ID' and len(val_list) == expected_len:
                data_for_df[key] = val_list

        # Create DataFrame
        df = pd.DataFrame(data_for_df, index=metadata['Sample_ID'])
        df.index.name = "Sample_ID"
        return df.reset_index()

    return pd.DataFrame()

test_series_matrix_file = 'GSE282742_series_matrix.txt'
metadata_df = extract_geo_metadata_to_dataframe(test_series_matrix_file)
metadata_df = metadata_df.rename(columns = {'Library_name' :'Sample_description'})
metadata_df = metadata_df[['Sample_description','disease_state','age','Sex']]
metadata_df = metadata_df.rename(columns={'Sample_description':'samples'})
metadata_df = metadata_df.set_index('samples')
print("--- Sample Metadata DataFrame ---")
display(metadata_df.head())
print(f"\nDataFrame shape: {metadata_df.shape}")

{'Sample_title': ['VGH0075: F, P-MCI, Subject ID: 289, Age: 69y', 'VGH0089: F, AD, Subject ID: 91, Age: 78y', 'VGH0146: M, P-MCI, Subject ID: 125, Age: 67y', 'VGH0195: F, AD, Subject ID: 323, Age: 73y', 'VGH0203: M, AD, Subject ID: 42, Age: 85y', 'VGH0216: M, AD, Subject ID: 16, Age: 83y', 'VGH0219: M, AD, Subject ID: 202, Age: 72y', 'VGH0240: F, S-MCI, Subject ID: 338, Age: 67y', 'VGH0252: F, S-MCI, Subject ID: 287, Age: 66y', 'VGH0264: F, AD, Subject ID: 347, Age: 81y', 'VGH0265: M, AD, Subject ID: 127, Age: 71y', 'VGH0293: M, P-MCI, Subject ID: 198, Age: 81y', 'VGH0309: M, AD, Subject ID: 256, Age: 84y', 'VGH0310: F, AD, Subject ID: 79, Age: 83y', 'VGH0321: M, S-MCI, Subject ID: 72, Age: 77y', 'VGH0329: M, S-MCI, Subject ID: 369, Age: 78y', 'VGH0402: M, S-MCI, Subject ID: 241, Age: 78y', 'VGH0415: M, S-MCI, Subject ID: 389, Age: 60y', 'VGH0459: F, S-MCI, Subject ID: 352, Age: 77y', 'VGH0515: F, S-MCI, Subject ID: 411, Age: 73y', 'VGH0516: M, AD, Subject ID: 260, Age: 62y', 'VGH0536:

,disease_state,age,Sex
samples,,,
VGH0075,P-MCI,69,F
VGH0089,AD,78,F
VGH0146,P-MCI,67,M
VGH0195,AD,73,F
VGH0203,AD,85,M



DataFrame shape: (116, 3)


In [160]:
# Merge genes and metadata
train_genes_metadata = pd.merge(
    metadata_df,
    normalized_train_df,
    left_index=True,   # Use the index of normalized_test_df
    right_index=True,  # Use the index of metadata_df
    how='inner'        # Keep only samples present in both
)
train_genes_metadata.head(5)

,disease_state,age,Sex,ENSG00000115590,ENSG00000198502,ENSG00000182557,ENSG00000260537,ENSG00000112759,ENSG00000205810,ENSG00000278588,...,ENSG00000184357,ENSG00000137959,ENSG00000197353,ENSG00000211945,ENSG00000270149,ENSG00000276410,ENSG00000275743,ENSG00000179044,ENSG00000244437,ENSG00000183134
VGH0075,P-MCI,69,F,7.328585,7.327867,2.510962,0.000000,4.150560,4.136684,0.000000,...,0.000000,7.244697,2.063503,4.263786,3.724650,0.000000,2.422233,2.990955,4.721919,2.906891
VGH0089,AD,78,F,7.129901,0.516015,1.286881,1.541019,2.217231,4.126808,0.367371,...,0.650765,3.695994,3.070389,2.885574,3.738768,0.887525,1.541019,1.664483,4.575917,1.070389
VGH0146,P-MCI,67,M,5.891176,3.903038,2.010780,0.782409,2.601697,3.732269,0.526069,...,0.286881,3.667892,3.786596,3.909773,2.017922,0.000000,2.831877,2.244887,4.754888,2.144046
VGH0195,AD,73,F,6.367196,4.372256,2.799087,0.000000,4.012569,2.641546,1.070389,...,1.389567,4.786074,4.426265,5.081936,2.356144,1.327687,3.145677,2.217231,5.572890,2.601697
VGH0203,AD,85,M,9.639413,3.112700,1.646163,2.000000,2.526069,2.117695,0.575312,...,0.565597,1.996389,1.049631,2.941106,2.695994,0.565597,3.375735,1.372952,3.764474,0.799087


In [161]:
# Save the DataFrame "train" to a CSV file
train_genes_metadata.to_csv('train_normalized_data.csv', index=True)

## Code for searching gene symbol

In [3]:
CORRECT_FILTER_NAME = 'link_ensembl_gene_id'

def map_ensembl_to_symbol_final(ensembl_ids):
    """
    Maps a list of Ensembl Gene IDs to their HGNC Gene Symbols using 
    the correct filter name ('gene_id').

    Args:
        ensembl_ids (list): A list of Ensembl Gene IDs (e.g., ['ENSG...']).

    Returns:
        pandas.DataFrame: A DataFrame with 'Ensembl_ID' and 'Gene_Symbol' columns.
    """
    try:
        # Use the default server which worked for filter discovery
        server = Server(host='http://www.ensembl.org')
    except Exception as e:
        print(f"Failed to connect to Ensembl server: {e}")
        return pd.DataFrame()

    # 2. Select the Mart and Dataset
    dataset = (server.marts['ENSEMBL_MART_ENSEMBL']
                     .datasets['hsapiens_gene_ensembl'])

    # 3. Define the desired attributes (output columns)
    attributes = ['ensembl_gene_id', 'hgnc_symbol']

    # 4. CRITICAL FIX: Use the discovered filter name 'gene_id'
    filters = {CORRECT_FILTER_NAME: ensembl_ids} 

    print(f"Querying Ensembl BioMart using filter '{CORRECT_FILTER_NAME}'...")
    
    try:
        # 5. Execute the query
        results_df = dataset.query(attributes=attributes, filters=filters)
        
        # Rename columns for clarity 
        results_df.columns = ['Ensembl_ID', 'Gene_Symbol']
        
        # Remove any rows where the Gene_Symbol is blank (unmapped IDs)
        results_df = results_df[results_df['Gene_Symbol'] != ''].reset_index(drop=True)
        
        print(f"Successfully retrieved mappings for {len(results_df)} IDs.")
        return results_df

    except Exception as e:
        print(f"An error occurred during the BioMart query: {e}")
        return pd.DataFrame()

Querying Ensembl BioMart using filter 'link_ensembl_gene_id'...
Successfully retrieved mappings for 4 IDs.

--- Final Mapping Results ---
        Ensembl_ID Gene_Symbol
0  ENSG00000100418       DESI1
1  ENSG00000105383        CD33
2  ENSG00000141510        TP53
3  ENSG00000288661         NaN
